Laplace Equation Using Gmsh

Imports

In [1]:
import gmsh
import numpy as np
from dolfinx.io import (VTXWriter, distribute_entity_data, gmshio)
from mpi4py import MPI
from petsc4py.PETSc import ScalarType
import numpy as np
import ufl
from dolfinx import fem, io, mesh, plot, nls, log, default_scalar_type
from dolfinx.fem.petsc import LinearProblem
from dolfinx.fem import (Constant, Function, functionspace,
                         assemble_scalar, dirichletbc, form, locate_dofs_topological, set_bc)
from ufl import ds, dx, grad, inner
from mpi4py import MPI
from petsc4py import PETSc
from basix.ufl import element
from dolfinx.mesh import create_mesh, meshtags_from_entities

Initialize Domian Boundary and Lengths

In [2]:
gmsh.initialize()

L = 30
H = 30
c_x = L/2
c_y = H/2
r = 1
gdim = 2
mesh_comm = MPI.COMM_WORLD
model_rank = 0
if mesh_comm.rank == model_rank:
    rectangle = gmsh.model.occ.addRectangle(0, 0, 0, L, H, tag=1)
    obstacle = gmsh.model.occ.addDisk(c_x, c_y, 0, r, r)

Meshing can only be done on a single core, use the zeroth (first) core to make the mesh

In [3]:
if mesh_comm.rank == model_rank:
    fluid = gmsh.model.occ.cut([(gdim, rectangle)], [(gdim, obstacle)])
    gmsh.model.occ.synchronize()

Add the ```fuild_marker``` which allows meshing of interior nodes 

In [4]:
fluid_marker = 1
if mesh_comm.rank == model_rank:
    volumes = gmsh.model.getEntities(dim=gdim)
    assert (len(volumes) == 1)
    gmsh.model.addPhysicalGroup(volumes[0][0], [volumes[0][1]], fluid_marker)
    gmsh.model.setPhysicalName(volumes[0][0], fluid_marker, "Fluid")

Add tags to the different surfaces

In [5]:
inlet_marker, outlet_marker, top_wall_marker, bottom_wall_marker, obstacle_marker = 2, 3, 4, 5, 6
inflow, outflow, topwall, bottomwall, obstacle = [], [], [], [], []
if mesh_comm.rank == model_rank:
    boundaries = gmsh.model.getBoundary(volumes, oriented=False)
    for boundary in boundaries:
        center_of_mass = gmsh.model.occ.getCenterOfMass(boundary[0], boundary[1])
        if np.allclose(center_of_mass, [0, H / 2, 0]):
            inflow.append(boundary[1])
        elif np.allclose(center_of_mass, [L, H / 2, 0]):
            outflow.append(boundary[1])
        elif np.allclose(center_of_mass, [L / 2, H, 0]):
            topwall.append(boundary[1])
        elif np.allclose(center_of_mass, [L / 2, 0, 0]):
            bottomwall.append(boundary[1])
        else:
            obstacle.append(boundary[1])
    gmsh.model.addPhysicalGroup(1, topwall, top_wall_marker)
    gmsh.model.setPhysicalName(1, top_wall_marker, "Top Wall")
    gmsh.model.addPhysicalGroup(1, bottomwall, bottom_wall_marker)
    gmsh.model.setPhysicalName(1, bottom_wall_marker, "Bottom Wall")
    gmsh.model.addPhysicalGroup(1, inflow, inlet_marker)
    gmsh.model.setPhysicalName(1, inlet_marker, "Inlet")
    gmsh.model.addPhysicalGroup(1, outflow, outlet_marker)
    gmsh.model.setPhysicalName(1, outlet_marker, "Outlet")
    gmsh.model.addPhysicalGroup(1, obstacle, obstacle_marker)
    gmsh.model.setPhysicalName(1, obstacle_marker, "Obstacle")

Mesh Refinement near the cylinder

In [6]:
# Create distance field from obstacle.
# Add threshold of mesh sizes based on the distance field
# LcMax -                  /--------
#                      /
# LcMin -o---------/
#        |         |       |
#       Point    DistMin DistMax
res_min = r
if mesh_comm.rank == model_rank:
    distance_field = gmsh.model.mesh.field.add("Distance")
    gmsh.model.mesh.field.setNumbers(distance_field, "EdgesList", obstacle)
    threshold_field = gmsh.model.mesh.field.add("Threshold")
    gmsh.model.mesh.field.setNumber(threshold_field, "IField", distance_field)
    gmsh.model.mesh.field.setNumber(threshold_field, "LcMin", res_min)
    gmsh.model.mesh.field.setNumber(threshold_field, "LcMax", 2*r)
    gmsh.model.mesh.field.setNumber(threshold_field, "DistMin", 0)
    gmsh.model.mesh.field.setNumber(threshold_field, "DistMax", 0)
    min_field = gmsh.model.mesh.field.add("Min")
    gmsh.model.mesh.field.setNumbers(min_field, "FieldsList", [threshold_field])
    gmsh.model.mesh.field.setAsBackgroundMesh(min_field)

Mesh the Domain

In [7]:
if mesh_comm.rank == model_rank:
    gmsh.option.setNumber("Mesh.Algorithm", 8)
    gmsh.option.setNumber("Mesh.RecombinationAlgorithm", 2)
    gmsh.option.setNumber("Mesh.RecombineAll", 1)
    gmsh.option.setNumber("Mesh.SubdivisionAlgorithm", 1)
    gmsh.model.mesh.generate(gdim)
    gmsh.model.mesh.setOrder(2)
    gmsh.model.mesh.optimize("Netgen")

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 5 (Ellipse)
Info    : [ 30%] Meshing curve 6 (Line)
Info    : [ 50%] Meshing curve 7 (Line)
Info    : [ 70%] Meshing curve 8 (Line)
Info    : [ 90%] Meshing curve 9 (Line)
Info    : Done meshing 1D (Wall 0.000385501s, CPU 0.000505s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay for Quads)
Info    : Simple recombination completed (Wall 0.00151521s, CPU 0.001452s): 97 quads, 18 triangles, 0 invalid quads, 0 quads with Q < 0.1, avg Q = 0.832706, min Q = 0.348838
Info    : Simple recombination completed (Wall 0.00399841s, CPU 0.003967s): 441 quads, 0 triangles, 0 invalid quads, 0 quads with Q < 0.1, avg Q = 0.869133, min Q = 0.500973
Info    : Done meshing 2D (Wall 0.00825625s, CPU 0.008211s)
Info    : Refining mesh...
Info    : Meshing order 2 (curvilinear on)...
Info    : [  0%] Meshing curve 5 order 2
Info    : [ 20%] Meshing curve 6 order 2
Info    : [ 40%] Meshing curve 7 order 2
Info    : [ 60%] Mes

Save the file to be viewed in gmsh

In [8]:
gmsh.write("LaplaceEquationFE_Practice_3.msh")

Info    : Writing 'LaplaceEquationFE_Practice_3.msh'...
Info    : Done writing 'LaplaceEquationFE_Practice_3.msh'


In [9]:
mesh, _, ft = gmshio.model_to_mesh(gmsh.model, mesh_comm, model_rank, gdim=gdim)
ft.name = "Facet markers"

Boundary Conditions

In [10]:
msh = element("Lagrange", mesh.topology.cell_name(), 2)
V = fem.functionspace(mesh, msh)
fdim = mesh.topology.dim - 1

# Define boundary conditions

# Inlet
class InletVelocity():
    def __call__(self, x):
        values = np.zeros((1, x.shape[1]), dtype=PETSc.ScalarType)
        values[0] = x[1] - L/2
        return values


u_inlet = fem.Function(V)
inlet_velocity = InletVelocity()
u_inlet.interpolate(inlet_velocity)
bc_inflow = dirichletbc(u_inlet, locate_dofs_topological(V, fdim, ft.find(inlet_marker)))

# Outlet
bc_outlet = dirichletbc(u_inlet, locate_dofs_topological(V, fdim, ft.find(outlet_marker)))

# Obstacle
bc_obstacle = dirichletbc(PETSc.ScalarType(0), locate_dofs_topological(V, fdim, ft.find(obstacle_marker)), V)

# Top Wall
bc_top_wall = dirichletbc(PETSc.ScalarType(L/2), locate_dofs_topological(V, fdim, ft.find(top_wall_marker)), V)

# Bottom Wall
bc_bottom_wall = dirichletbc(PETSc.ScalarType(-L/2), locate_dofs_topological(V, fdim, ft.find(bottom_wall_marker)), V)

Define v

In [11]:
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
f = fem.Constant(mesh,  default_scalar_type(0))
bcs = [bc_obstacle, bc_top_wall, bc_bottom_wall]
a = ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = f * v * ufl.dx

Solve the linear problem

In [12]:
problem = LinearProblem(a, L, bcs=bcs, petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

Export the data for post processing

In [13]:
xyz = V.tabulate_dof_coordinates()
x = xyz[:,0]
y = xyz[:,1]
print(x)
sol = np.array((uh.vector))
print(sol)
np.savetxt('FE_Practice_3_x_Mesh.csv', x, delimiter=',')
np.savetxt('FE_Practice_3_y_Mesh.csv', y, delimiter=',')
np.savetxt('FE_Practice_3_Sol.csv', sol, delimiter=',')

[3.00000000e+01 2.90625000e+01 3.00000000e+01 ... 4.68750000e-01
 7.31737658e-17 4.68038007e-01]
[ 15.          15.          14.05923181 ... -15.         -14.52960992
 -14.52938312]


In [15]:
from dolfinx.io import XDMFFile
from basix.ufl import element as VectorElement
with XDMFFile(MPI.COMM_WORLD, "LaplaceEquation2D_FE_Practice_3.xdmf", "w") as pfile_xdmf:
    uh.x.scatter_forward()
    P3 = VectorElement("Lagrange", mesh.basix_cell(), 2)
    u1 = Function(functionspace(mesh, P3))
    u1.interpolate(uh)
    pfile_xdmf.write_mesh(mesh)
    pfile_xdmf.write_function(u1)